# 🧬 Training pipeline: Training ranking model </span>

In this notebook, you will train a ranking model using gradient boosted trees. 

In [10]:
%load_ext autoreload
%autoreload 2

import warnings

warnings.filterwarnings("ignore")

from recsys.gcp.vertex_ai.serving.ranking import GCPRankingModel
from recsys.gcp.feature_store import client as fs_client
from recsys.gcp.bigquery import client as bq_client
from recsys.config import settings

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## ☁️ Connect to Vertex AI Feature Online Store

In [11]:
fs_client.initialize()
fos = fs_client.get_client()

2025-02-19 15:12:11.343 | INFO     | recsys.gcp.feature_store.client:get_client:30 - Retrieving Feature Store from us-central1/recsys-dev-gonzo-2/recsys_feature_store_dev


## 💿 Create training dataset

In [12]:
trans_fv, articles_fv, customers_f, rankings_fv = fs_client.get_feature_views(fos)

In [ ]:
rankings_df = bq_client.fetch_feature_view_data(
    feature_view=rankings_fv
)
trans_df = bq_client.fetch_feature_view_data(feature_view=trans_fv)
articles_df = bq_client.fetch_feature_view_data(
    feature_view=articles_fv,
    except_columns=["embeddings"]
)

2025-02-19 16:07:51.006 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:181 - Fetching data from feature view: rankings
2025-02-19 16:07:51.621 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:194 - Executing query: SELECT * EXCEPT(embeddings) FROM `recsys-dev-gonzo-2.recsys_dataset.recsys_rankings`


BadRequest: 400 Column embeddings in SELECT * EXCEPT list does not exist at [1:17]; reason: invalidQuery, location: query, message: Column embeddings in SELECT * EXCEPT list does not exist at [1:17]

Location: us-central1
Job ID: c96f2be1-29b3-4bf3-82df-9da08f3af8db


In [5]:
rankings_df = rankings_df.join(
    trans_df.select(["customer_id", "month_sin", "month_cos"]),
    on="customer_id",
    how="left",
)

In [6]:
rankings_df = rankings_df.rename({"perceived_colour_master_name": "colour_group_name"})
rankings_df.head(3)

customer_id,article_id,age,label,product_type_name,product_group_name,graphical_appearance_name,perceived_colour_value_name,colour_group_name,department_name,index_name,index_group_name,section_name,garment_group_name,month_sin,month_cos
str,str,f64,i64,str,str,str,str,str,str,str,str,str,str,f64,f64
"""b4397f1ef360771e411afa9b43ad9a…","""399061008""",66.0,0,"""Jacket""","""Garment Upper body""","""Denim""","""Dusty Light""","""Blue""","""Outwear""","""Ladieswear""","""Ladieswear""","""Womens Jackets""","""Outdoor""",-0.866025,0.5
"""b4397f1ef360771e411afa9b43ad9a…","""399061008""",66.0,0,"""Jacket""","""Garment Upper body""","""Denim""","""Dusty Light""","""Blue""","""Outwear""","""Ladieswear""","""Ladieswear""","""Womens Jackets""","""Outdoor""",1.0,6.1232e-17
"""b4397f1ef360771e411afa9b43ad9a…","""399061008""",66.0,0,"""Jacket""","""Garment Upper body""","""Denim""","""Dusty Light""","""Blue""","""Outwear""","""Ladieswear""","""Ladieswear""","""Womens Jackets""","""Outdoor""",1.0,6.1232e-17


In [7]:
X_train, X_val, y_train, y_val = utils.split_utils.train_test_split(
    df=rankings_df,
    test_size=settings.RANKING_DATASET_VALIDATION_SPLIT_SIZE,
    description="Ranking training dataset",
)

In [8]:
X_train.head(3)

customer_id,article_id,age,product_type_name,product_group_name,graphical_appearance_name,perceived_colour_value_name,colour_group_name,department_name,index_name,index_group_name,section_name,garment_group_name,month_sin,month_cos
str,str,f64,str,str,str,str,str,str,str,str,str,str,f64,f64
"""95d36d7665d46a9406653e204ea2f1…","""337991001""",52.0,"""Belt""","""Accessories""","""Solid""","""Dark""","""Black""","""Belts""","""Menswear""","""Menswear""","""Men Accessories""","""Accessories""",0.866025,0.5
"""78e1fc883f6bcff9bd576587e46dfa…","""571619001""",44.0,"""Swimsuit""","""Swimwear""","""Solid""","""Dark""","""Black""","""Swimwear""","""Lingeries/Tights""","""Ladieswear""","""Womens Swimwear, beachwear""","""Swimwear""",0.5,-0.866025
"""19ed12b3909e6d4fdb7e858c894d10…","""708258002""",34.0,"""Top""","""Garment Upper body""","""Embroidery""","""Medium Dusty""","""Pink""","""Tops Fancy Jersey""","""Divided""","""Divided""","""Divided Collection""","""Jersey Fancy""",-1.0,-1.8370e-16


In [9]:
y_train.head(3)

label
i64
0
0
0


# Training the ranking model

Let's train the ranking model:

In [10]:
model = training.ranking.RankingModelFactory.build()

In [11]:
trainer = training.ranking.RankingModelTrainer(
    model=model, train_dataset=(X_train, y_train), eval_dataset=(X_val, y_val)
)

In [12]:
trainer.fit()

[0]	validation_0-logloss:0.58784
[1]	validation_0-logloss:0.51518
[2]	validation_0-logloss:0.46404
[3]	validation_0-logloss:0.42282
[4]	validation_0-logloss:0.39191
[5]	validation_0-logloss:0.36765
[6]	validation_0-logloss:0.34559
[7]	validation_0-logloss:0.32857
[8]	validation_0-logloss:0.31820
[9]	validation_0-logloss:0.30925
[10]	validation_0-logloss:0.29643
[11]	validation_0-logloss:0.28544
[12]	validation_0-logloss:0.27934
[13]	validation_0-logloss:0.27337
[14]	validation_0-logloss:0.26739
[15]	validation_0-logloss:0.26209
[16]	validation_0-logloss:0.25876
[17]	validation_0-logloss:0.25397
[18]	validation_0-logloss:0.24803
[19]	validation_0-logloss:0.24429
[20]	validation_0-logloss:0.24154
[21]	validation_0-logloss:0.23634
[22]	validation_0-logloss:0.22959
[23]	validation_0-logloss:0.22698
[24]	validation_0-logloss:0.22474
[25]	validation_0-logloss:0.22081
[26]	validation_0-logloss:0.21835
[27]	validation_0-logloss:0.21443
[28]	validation_0-logloss:0.21076
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=5,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.2, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

## Evaluating the ranking model

Next, you'll evaluate how well the model performs on the validation data using metrics for classification such as precision, recall and f1-score:

In [13]:
metrics = trainer.evaluate(log=True)

2025-02-18 12:32:40.242 | INFO     | recsys.training.ranking:evaluate:111 -               precision    recall  f1-score   support

           0       1.00      0.97      0.98   1669305
           1       0.75      0.99      0.85    166480

    accuracy                           0.97   1835785
   macro avg       0.87      0.98      0.92   1835785
weighted avg       0.98      0.97      0.97   1835785



It can be seen that the model has a low F1-score on the positive class (higher is better). The performance could potentially be improved by adding more features to the dataset, e.g. image embeddings.

Let's see which features your model considers important.

In [14]:
trainer.get_feature_importance()

{'customer_id': 0.3297032,
 'age': 0.21363337,
 'index_name': 0.048178244,
 'index_group_name': 0.047474615,
 'section_name': 0.0384507,
 'product_group_name': 0.037767608,
 'month_cos': 0.037423663,
 'month_sin': 0.037174065,
 'department_name': 0.03405898,
 'garment_group_name': 0.032587793,
 'product_type_name': 0.03167115,
 'graphical_appearance_name': 0.029043257,
 'article_id': 0.028192334,
 'perceived_colour_value_name': 0.028049644,
 'colour_group_name': 0.026591362}

# 🗄️ Upload models to Vertex AI model registry

In [15]:
ranking_model_gcp = GCPRankingModel(model=model)

In [16]:
ranking_local_path = ranking_model_gcp.save_to_local("ranking_model")

In [17]:
ranking_model_vertex = ranking_model_gcp.upload_to_vertex_ai(
    model_name="ranking_model_v1",
    description="Ranking model for two-tower recommender system",
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.2-1:latest",
)

2025-02-18 12:32:40.381 | INFO     | recsys.gcp_integrations.model_registry:upload_model_to_registry:25 - Saving model into: /tmp/ranking_model_v1
2025-02-18 12:32:40.388 | INFO     | recsys.gcp_integrations.model_registry:upload_model_to_registry:44 - Uploading model to ranking_model_v1 to Vertex AI


Creating Model
Create Model backing LRO: projects/99924275580/locations/us-central1/models/4814397479682310144/operations/8501784442698727424
Model created. Resource name: projects/99924275580/locations/us-central1/models/4814397479682310144@1
To use this Model in another session:
model = aiplatform.Model('projects/99924275580/locations/us-central1/models/4814397479682310144@1')


2025-02-18 12:32:46.871 | INFO     | recsys.gcp_integrations.model_registry:upload_model_to_registry:54 - Model uploaded with resource name: projects/99924275580/locations/us-central1/models/4814397479682310144


In [18]:
rankind_endpoint = ranking_model_gcp.deploy_endpoint(
    model=ranking_model_vertex,
    endpoint_id="recsys-rankings-model",
)

2025-02-18 12:32:47.687 | INFO     | recsys.gcp_integrations.two_tower_serving:deploy_endpoint:70 - Found existing endpoint: recsys-rankings-model


Deploying model to Endpoint : projects/99924275580/locations/us-central1/endpoints/recsys-rankings-model
Deploy Endpoint model backing LRO: projects/99924275580/locations/us-central1/endpoints/recsys-rankings-model/operations/8751734222017789952
Endpoint model deployed. Resource name: projects/99924275580/locations/us-central1/endpoints/recsys-rankings-model


2025-02-18 12:36:41.488 | INFO     | recsys.gcp_integrations.two_tower_serving:deploy_endpoint:83 - Model deployed to endpoint: projects/99924275580/locations/us-central1/endpoints/recsys-rankings-model
